In [1]:
from __future__ import print_function
import six
import sys
import os
from optparse import OptionParser
from keras.models import load_model, Model, model_from_json
from argparse import ArgumentParser
from keras import backend as K
from keras.layers import Input
import numpy as np
# fix random seed for reproducibility
seed = 42
np.random.seed(seed)
import h5py
import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc
import pandas as pd
from keras.optimizers import Adam, Nadam
from callbacks import all_callbacks
import pandas as pd
from keras.layers import Input
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
import yaml
from train import parse_config, get_features

Using TensorFlow backend.


In [2]:
from collections import namedtuple
Option = namedtuple("MyStruct", "inputFile tree config outputDir")

options = Option(
    inputFile = '../data/processed-pythia82-lhc13-all-pt1-50k-r1_h022_e0175_t220_nonu_withPars_truth.z',
    tree = 't_allpar_new',
    config = 'train_config_lstm.yml',
    outputDir = '../lstm_weights/'
)

print("Loading configuration from", options.config)
config = open(options.config, 'r')
yamlConfig =  yaml.load(config, Loader=yaml.FullLoader)
print(yamlConfig)

Loading configuration from train_config_lstm.yml
{'Inputs': ['j1_ptrel', 'j1_etarot', 'j1_phirot', 'j1_erel', 'j1_deltaR', 'j1_pdgid', 'j_index'], 'Labels': ['j_g', 'j_q', 'j_w', 'j_z', 'j_t', 'j_index'], 'KerasModel': 'lstm_model', 'KerasModelRetrain': 'lstm_model_constraint', 'KerasLoss': 'categorical_crossentropy', 'L1Reg': 0.0001, 'L1RegR': 0.001, 'NormalizeInputs': 1, 'InputType': 'Conv1D', 'MaxParticles': 20}


In [3]:
X_train_val, X_test, y_train_val, y_test, labels  = get_features(options, yamlConfig)

Keys: <KeysViewHDF5 ['t_allpar_new']>
load 24816
load 0
load 1000
load 2000
load 3000
load 4000
load 5000
load 6000
load 7000
load 8000
load 9000
load 10000
load 11000
load 12000
load 13000
load 14000
load 15000
load 16000
load 17000
load 18000
load 19000
load 20000
load 21000
load 22000
load 23000
load 24000
(24816, 20, 6)
(19852, 20, 6)
(4964, 20, 6)


In [8]:
from models.models import lstm_model
keras_model = lstm_model(Input(shape=X_train_val.shape[1:]), y_train_val.shape[1], l1Reg=yamlConfig['L1Reg'] )

startlearningrate=0.0001
adam = Adam(lr=startlearningrate)
keras_model.compile(optimizer=adam, loss=[yamlConfig['KerasLoss']], metrics=['accuracy'])


callbacks=all_callbacks(stop_patience=1000, 
                        lr_factor=0.5,
                        lr_patience=10,
                        lr_epsilon=0.000001, 
                        lr_cooldown=2, 
                        lr_minimum=0.0000001,
                        outputDir=options.outputDir)

keras_model.fit(X_train_val, y_train_val, batch_size = 1024, epochs = 100,
                validation_split = 0.25, shuffle = True, callbacks = callbacks.callbacks)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 20, 6)             0         
_________________________________________________________________
lstm_lstm (LSTM)             (None, 16)                1472      
_________________________________________________________________
dropout_2 (Dropout)          (None, 16)                0         
_________________________________________________________________
rnn_densef (Dense)           (None, 5)                 85        
Total params: 1,557
Trainable params: 1,557
Non-trainable params: 0
_________________________________________________________________
None
Train on 14889 samples, validate on 4963 samples
Epoch 1/100
14889/14889 [==============================] - 3s 213us/step - loss: 1.8763 - acc: 0.0706 - val_loss: 1.8005 - val_acc: 0.0711

***callbacks***
saving losses to ../lstm_weights/losses.log

Epoch 00001: val_


***callbacks end***

Epoch 13/100
14889/14889 [==============================] - 1s 50us/step - loss: 1.3249 - acc: 0.2310 - val_loss: 1.2838 - val_acc: 0.2053

***callbacks***
saving losses to ../lstm_weights/losses.log

Epoch 00013: val_loss improved from 1.31929 to 1.28377, saving model to ../lstm_weights//KERAS_check_best_model.h5

Epoch 00013: val_loss improved from 1.31929 to 1.28377, saving model to ../lstm_weights//KERAS_check_best_model_weights.h5

Epoch 00013: saving model to ../lstm_weights//KERAS_check_model_last.h5

Epoch 00013: saving model to ../lstm_weights//KERAS_check_model_last_weights.h5

***callbacks end***

Epoch 14/100
14889/14889 [==============================] - 1s 38us/step - loss: 1.2823 - acc: 0.2633 - val_loss: 1.2474 - val_acc: 0.2406

***callbacks***
saving losses to ../lstm_weights/losses.log

Epoch 00014: val_loss improved from 1.28377 to 1.24742, saving model to ../lstm_weights//KERAS_check_best_model.h5

Epoch 00014: val_loss improved from 1.28377 t

14889/14889 [==============================] - 1s 44us/step - loss: 0.6756 - acc: 0.8144 - val_loss: 0.5890 - val_acc: 0.8811

***callbacks***
saving losses to ../lstm_weights/losses.log

Epoch 00026: val_loss improved from 0.66822 to 0.58895, saving model to ../lstm_weights//KERAS_check_best_model.h5

Epoch 00026: val_loss improved from 0.66822 to 0.58895, saving model to ../lstm_weights//KERAS_check_best_model_weights.h5

Epoch 00026: saving model to ../lstm_weights//KERAS_check_model_last.h5

Epoch 00026: saving model to ../lstm_weights//KERAS_check_model_last_weights.h5

***callbacks end***

Epoch 27/100
14889/14889 [==============================] - 1s 52us/step - loss: 0.5944 - acc: 0.8461 - val_loss: 0.5065 - val_acc: 0.9039

***callbacks***
saving losses to ../lstm_weights/losses.log

Epoch 00027: val_loss improved from 0.58895 to 0.50654, saving model to ../lstm_weights//KERAS_check_best_model.h5

Epoch 00027: val_loss improved from 0.58895 to 0.50654, saving model to ../lstm_


***callbacks***
saving losses to ../lstm_weights/losses.log

Epoch 00039: val_loss improved from 0.18667 to 0.18172, saving model to ../lstm_weights//KERAS_check_best_model.h5

Epoch 00039: val_loss improved from 0.18667 to 0.18172, saving model to ../lstm_weights//KERAS_check_best_model_weights.h5

Epoch 00039: saving model to ../lstm_weights//KERAS_check_model_last.h5

Epoch 00039: saving model to ../lstm_weights//KERAS_check_model_last_weights.h5

***callbacks end***

Epoch 40/100
14889/14889 [==============================] - 1s 34us/step - loss: 0.2303 - acc: 0.9219 - val_loss: 0.1777 - val_acc: 0.9430

***callbacks***
saving losses to ../lstm_weights/losses.log

Epoch 00040: val_loss improved from 0.18172 to 0.17770, saving model to ../lstm_weights//KERAS_check_best_model.h5

Epoch 00040: val_loss improved from 0.18172 to 0.17770, saving model to ../lstm_weights//KERAS_check_best_model_weights.h5

Epoch 00040: saving model to ../lstm_weights//KERAS_check_model_last.h5

Epoch 000


***callbacks end***

Epoch 53/100
14889/14889 [==============================] - 1s 34us/step - loss: 0.1727 - acc: 0.9425 - val_loss: 0.1344 - val_acc: 0.9603

***callbacks***
saving losses to ../lstm_weights/losses.log

Epoch 00053: val_loss improved from 0.13713 to 0.13440, saving model to ../lstm_weights//KERAS_check_best_model.h5

Epoch 00053: val_loss improved from 0.13713 to 0.13440, saving model to ../lstm_weights//KERAS_check_best_model_weights.h5

Epoch 00053: saving model to ../lstm_weights//KERAS_check_model_last.h5

Epoch 00053: saving model to ../lstm_weights//KERAS_check_model_last_weights.h5

***callbacks end***

Epoch 54/100
14889/14889 [==============================] - 1s 36us/step - loss: 0.1692 - acc: 0.9424 - val_loss: 0.1317 - val_acc: 0.9611

***callbacks***
saving losses to ../lstm_weights/losses.log

Epoch 00054: val_loss improved from 0.13440 to 0.13172, saving model to ../lstm_weights//KERAS_check_best_model.h5

Epoch 00054: val_loss improved from 0.13440 t

14889/14889 [==============================] - 0s 33us/step - loss: 0.1268 - acc: 0.9641 - val_loss: 0.1009 - val_acc: 0.9744

***callbacks***
saving losses to ../lstm_weights/losses.log

Epoch 00066: val_loss improved from 0.10262 to 0.10092, saving model to ../lstm_weights//KERAS_check_best_model.h5

Epoch 00066: val_loss improved from 0.10262 to 0.10092, saving model to ../lstm_weights//KERAS_check_best_model_weights.h5

Epoch 00066: saving model to ../lstm_weights//KERAS_check_model_last.h5

Epoch 00066: saving model to ../lstm_weights//KERAS_check_model_last_weights.h5

***callbacks end***

Epoch 67/100
14889/14889 [==============================] - 0s 26us/step - loss: 0.1293 - acc: 0.9630 - val_loss: 0.0978 - val_acc: 0.9746

***callbacks***
saving losses to ../lstm_weights/losses.log

Epoch 00067: val_loss improved from 0.10092 to 0.09784, saving model to ../lstm_weights//KERAS_check_best_model.h5

Epoch 00067: val_loss improved from 0.10092 to 0.09784, saving model to ../lstm_


***callbacks***
saving losses to ../lstm_weights/losses.log

Epoch 00079: val_loss improved from 0.07960 to 0.07807, saving model to ../lstm_weights//KERAS_check_best_model.h5

Epoch 00079: val_loss improved from 0.07960 to 0.07807, saving model to ../lstm_weights//KERAS_check_best_model_weights.h5

Epoch 00079: saving model to ../lstm_weights//KERAS_check_model_last.h5

Epoch 00079: saving model to ../lstm_weights//KERAS_check_model_last_weights.h5

***callbacks end***

Epoch 80/100
14889/14889 [==============================] - 0s 32us/step - loss: 0.0923 - acc: 0.9741 - val_loss: 0.0769 - val_acc: 0.9776

***callbacks***
saving losses to ../lstm_weights/losses.log

Epoch 00080: val_loss improved from 0.07807 to 0.07688, saving model to ../lstm_weights//KERAS_check_best_model.h5

Epoch 00080: val_loss improved from 0.07807 to 0.07688, saving model to ../lstm_weights//KERAS_check_best_model_weights.h5

Epoch 00080: saving model to ../lstm_weights//KERAS_check_model_last.h5

Epoch 000

14889/14889 [==============================] - 0s 23us/step - loss: 0.0787 - acc: 0.9796 - val_loss: 0.0648 - val_acc: 0.9827

***callbacks***
saving losses to ../lstm_weights/losses.log

Epoch 00093: val_loss improved from 0.06570 to 0.06479, saving model to ../lstm_weights//KERAS_check_best_model.h5

Epoch 00093: val_loss improved from 0.06570 to 0.06479, saving model to ../lstm_weights//KERAS_check_best_model_weights.h5

Epoch 00093: saving model to ../lstm_weights//KERAS_check_model_last.h5

Epoch 00093: saving model to ../lstm_weights//KERAS_check_model_last_weights.h5

***callbacks end***

Epoch 94/100
14889/14889 [==============================] - 0s 22us/step - loss: 0.0787 - acc: 0.9795 - val_loss: 0.0640 - val_acc: 0.9833

***callbacks***
saving losses to ../lstm_weights/losses.log

Epoch 00094: val_loss improved from 0.06479 to 0.06401, saving model to ../lstm_weights//KERAS_check_best_model.h5

Epoch 00094: val_loss improved from 0.06479 to 0.06401, saving model to ../lstm_

In [6]:
with open("../lstm_weights/model.json", "w") as json_file:
    json_file.write(keras_model.to_json())